In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re

#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

In [2]:
train = pd.read_csv('zip/train2.csv.gz', index_col=['id'], compression='gzip')
train.head()

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,last_word_title,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr
id,,,,,,,,,,,,,,,,,,,,,
2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,angle bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.500000,0.500000,...,0,0,0.833333,0.011236,0.500000,0.000000,0.0,0.0,6,89
3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,l bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.000000,0.000000,...,0,0,1.000000,1.000000,0.000000,0.000000,0.0,0.0,6,89
9,100002,behr premium textur deckov 1 gal. sc 141 tugbo...,deck,3.00,deck over,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,behr premium textur deckov innov solid color c...,BEHR Premium Textured DECKOVER is an innovativ...,0.000000,0.000000,...,1,1,1.000000,1.000000,0.000000,0.500000,0.0,0.0,12,122
16,100005,delta vero 1 handl shower faucet trim kit chro...,rain shower head,2.33,rain shower head,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,updat bathroom delta vero single-handl shower ...,Update your bathroom with the Delta Vero Singl...,0.333333,0.333333,...,0,0,0.363636,0.073529,0.333333,0.000000,0.0,0.0,11,68
17,100005,delta vero 1 handl shower faucet trim kit chro...,shower faucet,2.67,shower only faucet,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,updat bathroom delta vero single-handl shower ...,Update your bathroom with the Delta Vero Singl...,1.000000,1.000000,...,1,1,0.363636,0.073529,0.666667,0.333333,0.0,0.0,11,68


In [3]:
test = pd.read_csv('zip/test2.csv.gz', index_col=['id'], compression='gzip')
test.head()

,product_uid,product_title,search_term,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,len_search_term_words,...,last_word_title,last_word_descr,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,prod_title_ind,descr_ind
id,,,,,,,,,,,,,,,,,,,,,
1,100001,simpson strong tie 12 gaug angl,90deg. bracket,90 degree bracket,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.000000,0.000000,2,...,0,0,0.000000,0.000000,0.0,0.0,6,89,1.0,1.00000
4,100001,simpson strong tie 12 gaug angl,metal l bracket,metal l brackets,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.000000,0.000000,3,...,0,0,0.000000,0.000000,0.0,0.0,6,89,1.0,1.00000
5,100001,simpson strong tie 12 gaug angl,simpson sku abl,simpson sku able,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.333333,0.333333,3,...,0,0,0.333333,0.333333,0.0,0.0,6,89,0.0,0.11236
6,100001,simpson strong tie 12 gaug angl,simpson strong tie,simpson strong ties,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",1.000000,0.333333,3,...,1,0,0.666667,0.666667,0.0,0.0,6,89,0.0,0.11236
7,100001,simpson strong tie 12 gaug angl,simpson strong tie hcc668,simpson strong tie hcc668,Simpson Strong-Tie 12-Gauge Angle,"not angl make joint stronger, also provid cons...","Not only do angles make joints stronger, they ...",0.750000,0.250000,4,...,0,0,0.500000,0.750000,0.0,0.0,6,89,0.0,0.11236


In [11]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

    100% |████████████████████████████████| 51kB 435kB/s ta 0:00:01
  Running setup.py bdist_wheel for python-Levenshtein ... done
  Stored in directory: /home/eugene/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [7]:
from fuzzywuzzy import fuzz

@np.vectorize
def fuzzy_match(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.token_set_ratio(search_term, field)

In [8]:
train['fuzzy_title'] = fuzzy_match(train['search_term'], train['product_title'])
train['fuzzy_descr'] = fuzzy_match(train['search_term'], train['descr'])

train['fuzzy__orig_title'] = fuzzy_match(train['search_term'], train['orig_product_title'])
train['fuzzy__orig_descr'] = fuzzy_match(train['search_term'], train['orig_descr'])

In [2]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

In [3]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy__orig_title', 'fuzzy__orig_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

In [4]:
class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, train):
        return train[tree_cols].values


class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, x, y=None):
        return self
    def transform(self, dataset):
        return dataset[self.col].apply(str)

In [6]:
clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
clf.fit(train[use_cols])
X_train = clf.transform(train[use_cols])

In [ ]:
test['fuzzy_title'] = fuzzy_match(test['search_term'], test['product_title'])
test['fuzzy_descr'] = fuzzy_match(test['search_term'], test['descr'])

test['fuzzy__orig_title'] = fuzzy_match(test['search_term'], test['orig_product_title'])
test['fuzzy__orig_descr'] = fuzzy_match(test['search_term'], test['orig_descr'])

In [7]:
X_test = clf.transform(test[use_cols])

In [14]:
y = train['relevance'].astype(np.float).values

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5)
params = {
    'num_leaves': [70, 80],
    'n_estimators': [600, 700, 800]
}
model = GridSearchCV(gbm, params,
                     cv=4,
                     n_jobs=1,
                     return_train_score=False,
                     verbose=2)
model.fit(X_train, y)
grid_df = pd.DataFrame(model.cv_results_)
grid_df.drop(columns=grid_df.columns[:3], inplace=True)
grid_df.sort_values(['rank_test_score'], inplace=True)
grid_df.head()

Fitting 4 folds for each of 6 candidates, totalling 24 fits
[CV] n_estimators=600, num_leaves=70 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................. n_estimators=600, num_leaves=70, total=  12.2s
[CV] n_estimators=600, num_leaves=70 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.2s remaining:    0.0s


[CV] .................. n_estimators=600, num_leaves=70, total=  12.2s
[CV] n_estimators=600, num_leaves=70 .................................
[CV] .................. n_estimators=600, num_leaves=70, total=  12.2s
[CV] n_estimators=600, num_leaves=70 .................................
[CV] .................. n_estimators=600, num_leaves=70, total=  12.1s
[CV] n_estimators=600, num_leaves=80 .................................
[CV] .................. n_estimators=600, num_leaves=80, total=  13.9s
[CV] n_estimators=600, num_leaves=80 .................................
[CV] .................. n_estimators=600, num_leaves=80, total=  13.3s
[CV] n_estimators=600, num_leaves=80 .................................
[CV] .................. n_estimators=600, num_leaves=80, total=  13.5s
[CV] n_estimators=600, num_leaves=80 .................................
[CV] .................. n_estimators=600, num_leaves=80, total=  13.2s
[CV] n_estimators=700, num_leaves=70 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  5.8min finished


,std_score_time,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
5,0.001250,800,80,"{'n_estimators': 800, 'num_leaves': 80}",0.264430,0.279296,0.291520,0.144581,0.244958,0.058740,1
3,0.002289,700,80,"{'n_estimators': 700, 'num_leaves': 80}",0.263633,0.277954,0.290620,0.143319,0.243883,0.058839,2
4,0.002318,800,70,"{'n_estimators': 800, 'num_leaves': 70}",0.264098,0.277655,0.290912,0.141508,0.243545,0.059667,3
2,0.001510,700,70,"{'n_estimators': 700, 'num_leaves': 70}",0.263067,0.275872,0.289493,0.140669,0.242277,0.059401,4
1,0.001181,600,80,"{'n_estimators': 600, 'num_leaves': 80}",0.262223,0.275599,0.288926,0.141183,0.241984,0.058957,5


In [17]:
y_pred = model.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.46969

In [19]:
tc_len = len(tree_cols)
print(dict(zip(tree_cols, model.best_estimator_.feature_importances_[:tc_len]) ))

{'match_product_title': 244, 'match_descr': 154, 'len_search_term_words': 209, 'len_search_term_letters': 508, 'match_pos_title': 173, 'match_brand': 42, 'match_pos_descr': 133, 'prod_title_ind': 445, 'descr_ind': 402, 'words_std_title': 185, 'words_std_descr': 213, 'len_title': 356, 'len_descr': 270, 'last_word_title': 80, 'last_word_descr': 62, 'match_orig_title': 134, 'match_orig_descr': 77, 'fuzzy_title': 540, 'fuzzy_descr': 338, 'fuzzy__orig_title': 457, 'fuzzy__orig_descr': 238}


In [20]:
train.to_csv('zip/train2.csv.gz', compression='gzip')
test.to_csv('zip/test2.csv.gz', compression='gzip')


In [33]:
train = pd.read_csv('zip/train2.csv.gz', index_col=['id'], compression='gzip')
test = pd.read_csv('zip/test2.csv.gz', index_col=['id'], compression='gzip')

In [12]:
y = train['relevance'].astype(np.float).values

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=800)

gbm.fit(X_train, y)
gbm.score(X_train, y)

0.6003340835524165

In [13]:
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.46959

In [34]:
train = train.drop(columns=['fuzzy__orig_title', 'fuzzy__orig_descr'])
train['fuzzy_orig_title'] = fuzzy_match(train['orig_search_term'], train['orig_product_title'])
train['fuzzy_orig_descr'] = fuzzy_match(train['orig_search_term'], train['orig_descr'])
train.to_csv('train3.csv')
test = test.drop(columns=['fuzzy__orig_title', 'fuzzy__orig_descr'])
test['fuzzy_orig_title'] = fuzzy_match(test['orig_search_term'], test['orig_product_title'])
test['fuzzy_orig_descr'] = fuzzy_match(test['orig_search_term'], test['orig_descr'])
test.to_csv('test3.csv')

In [3]:

train = pd.read_csv('zip/train3.csv.gz', index_col=['id'], compression='gzip')
test = pd.read_csv('zip/test3.csv.gz', index_col=['id'], compression='gzip')

In [4]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

In [9]:
clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
clf.fit(train[use_cols])
X_train = clf.transform(train[use_cols])

In [10]:

X_test = clf.transform(test[use_cols])

In [12]:
y = train['relevance'].astype(np.float).values

In [11]:
gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80)

params = {
    'n_estimators': [800, 900, 1000]
}
model = GridSearchCV(gbm, params,
                     cv=4,
                     n_jobs=1,
                     return_train_score=False,
                     verbose=2)
model.fit(X_train, y)
grid_df = pd.DataFrame(model.cv_results_)
grid_df.drop(columns=grid_df.columns[:3], inplace=True)
grid_df.sort_values(['rank_test_score'], inplace=True)
grid_df.head()

Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] n_estimators=800 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................. n_estimators=800, total=  17.5s
[CV] n_estimators=800 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.5s remaining:    0.0s


[CV] ................................. n_estimators=800, total=  17.2s
[CV] n_estimators=800 ................................................
[CV] ................................. n_estimators=800, total=  17.0s
[CV] n_estimators=800 ................................................
[CV] ................................. n_estimators=800, total=  17.0s
[CV] n_estimators=900 ................................................
[CV] ................................. n_estimators=900, total=  19.3s
[CV] n_estimators=900 ................................................
[CV] ................................. n_estimators=900, total=  19.0s
[CV] n_estimators=900 ................................................
[CV] ................................. n_estimators=900, total=  18.8s
[CV] n_estimators=900 ................................................
[CV] ................................. n_estimators=900, total=  18.6s
[CV] n_estimators=1000 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  3.8min finished


,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
2,0.002590,1000,{'n_estimators': 1000},0.263360,0.278008,0.293740,0.147743,0.245714,0.057573,1
1,0.002163,900,{'n_estimators': 900},0.262952,0.277007,0.293087,0.146782,0.244959,0.057675,2
0,0.002775,800,{'n_estimators': 800},0.262467,0.275620,0.291925,0.145751,0.243942,0.057641,3


In [12]:
y_pred = model.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

In [13]:
train['pred_diff'] = train['relevance'] - model.predict(X_train)
pd.set_option('display.max_colwidth', -1)
train.sort_values(['pred_diff'], ascending=False)

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,pred_diff
id,,,,,,,,,,,,,,,,,,,,,
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510replaces: dantona 3aa, empir cpb-400, general electr bt-17, panason p-p341pa, recoton t121, uniden ba-3003.6-volt / 1200mahnickel-met hydride1 year warranti","Power your Panasonic, American Telecom, Tele-Phone, Key Phone, Audiovox, Uniden, Universal, and Zeta cordless phones with Lenmar's Nickel-Metal Hydride cordless phone replacement battery. This 3.6-Volt/1200mAh battery will provide reliable, long-lasting power. Compatible with Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, and Zeta 510.Fits: Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, Zeta 510Replaces: Dantona 3AA, Empire CPB-400, General Electric BT-17, Panasonic P-P341PA, Recoton T121, Uniden BA-3003.6-volt / 1200mAhNickel-metal hydride1 year warranty",0.666667,0.333333,...,0.666667,0.000000,0.000000,11,82,79,52,74,74,1.117467
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,0.500000,0.000000,0.250000,11,95,60,60,57,57,1.107107
193001,183692,lichtenberg white no. 918 millenni ryan heather textur sheer curtain panel 40in. w xbi 95in. l,w g 918,3.00,w g 918,"LICHTENBERG White No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. W x 9

In [14]:

train.sort_values(['pred_diff'], ascending=True)

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,pred_diff
id,,,,,,,,,,,,,,,,,,,,,
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"whether boil pasta, make batch chili cook grains, large, 10 quart stockpot right size. when stovetop, two short side handl help take less space cabinet. the hard-anod construct prefer even heat distribut throughout pan sides. the nonstick cook surfac mean need worri food stick scrub dirti pan - cleanup breeze. in fact, rachael stockpot dishwash safe ad convenience.hard-anod cookwar except durable, plus heat quick evenly, reduc ""hot spots"" burn foodsa clear coat exterior make cookwar dishwash safe conveniencelong-last nonstick interior let food slide eas make cleanup quick easyfun rubber handl provid comfort grip oven safe 350 degre fahrenheittemp glass lid watch food cook without remov lid keep heat moistur lock inlifetim limit warranti","Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience.Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foodsA clear coat exterior makes this cookware dishwasher safe for convenienceLong-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easyFun rubberized handles provide a comfortable grip and are oven safe to 350 degrees FahrenheitTempered glass lid to watch your foods cook without having to remove the lid to keep heat and moisture locked inLifetime limited warranty",1.000000,0.500000,...,0.500000,0.000000,0.000000,6,113,100,78,100,78,-1.508317
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"enhanc home romano boxwood spiral topiari tree. this wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. you alway look forward rich green tone romano boxwood spiral topiari tree home. if look high qualiti artifici tree make bold statement home, it.light mobil designov 700 leavesindoor/outdooriron frameuv resist rubber leaves7 in. potenh room hous","Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",1.000000,1.000000,...,1.000000,0.000000,0.500000,6,57,100,100,100,100,-1.434881
156918,160006,werner 14ft. fiberglass round rung straight ladder 375lb. load capac type iaa duti rate,14ft. ladder,1.00,14 ft ladder,Werner 14 ft. Fiberglass Round Rung Straight Ladder with 375 lb. Load Capacity Type IAA Duty Rating,"the 7114-1 one-sect round rung 14 ft. straight ladder made non-conduct fiberglass rails. easi transport maneuver, ladder bear duti rate 375 lbs. make durabl long lasting. the slip-resistant, traction-tr rung round, durabl rail shield shoe bracket help protect rail damage. the plate rung assembl rivet four point side rails.alflo twist-proof perform

In [5]:
@np.vectorize
def fuzzy_match_partial(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.partial_token_set_ratio(search_term, field)

In [16]:
train['fuzzy_title'] = fuzzy_match_partial(train['search_term'], train['product_title'])
train['fuzzy_descr'] = fuzzy_match_partial(train['search_term'], train['descr'])

train['fuzzy_orig_title'] = fuzzy_match_partial(train['orig_search_term'], train['orig_product_title'])
train['fuzzy_orig_descr'] = fuzzy_match_partial(train['orig_search_term'], train['orig_descr'])

In [17]:
test['fuzzy_title'] = fuzzy_match_partial(test['search_term'], test['product_title'])
test['fuzzy_descr'] = fuzzy_match_partial(test['search_term'], test['descr'])

test['fuzzy_orig_title'] = fuzzy_match_partial(test['orig_search_term'], test['orig_product_title'])
test['fuzzy_orig_descr'] = fuzzy_match_partial(test['orig_search_term'], test['orig_descr'])

In [18]:
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

In [20]:
gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80)
params = {
    'n_estimators': [1000]
}
model = GridSearchCV(gbm, params,
                     cv=4,
                     n_jobs=1,
                     return_train_score=False,
                     verbose=2)
model.fit(X_train, y)
grid_df = pd.DataFrame(model.cv_results_)
grid_df.drop(columns=grid_df.columns[:3], inplace=True)
grid_df.sort_values(['rank_test_score'], inplace=True)
grid_df.head()

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[CV] n_estimators=1000 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................ n_estimators=1000, total=  20.7s
[CV] n_estimators=1000 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.7s remaining:    0.0s


[CV] ................................ n_estimators=1000, total=  20.5s
[CV] n_estimators=1000 ...............................................
[CV] ................................ n_estimators=1000, total=  20.5s
[CV] n_estimators=1000 ...............................................
[CV] ................................ n_estimators=1000, total=  20.2s


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.4min finished


,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005336,1000,{'n_estimators': 1000},0.264056,0.280212,0.290932,0.139865,0.243768,0.060745,1


In [21]:
y_pred = model.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.47003

In [22]:
tc_len = len(tree_cols)
print(dict(zip(tree_cols, model.best_estimator_.feature_importances_[:tc_len]) ))

{'match_product_title': 453, 'match_descr': 240, 'len_search_term_words': 244, 'len_search_term_letters': 635, 'match_pos_title': 229, 'match_brand': 69, 'match_pos_descr': 226, 'prod_title_ind': 543, 'descr_ind': 500, 'words_std_title': 288, 'words_std_descr': 274, 'len_title': 404, 'len_descr': 353, 'last_word_title': 94, 'last_word_descr': 71, 'match_orig_title': 145, 'match_orig_descr': 135, 'fuzzy_title': 216, 'fuzzy_descr': 170, 'fuzzy_orig_title': 208, 'fuzzy_orig_descr': 91}


In [8]:
train = pd.read_csv('zip/train3.csv.gz', index_col=['id'], compression='gzip')
test = pd.read_csv('zip/test3.csv.gz', index_col=['id'], compression='gzip')

## train
train['part_fuzzy_title'] = fuzzy_match_partial(train['search_term'], train['product_title'])
train['part_fuzzy_descr'] = fuzzy_match_partial(train['search_term'], train['descr'])

train['part_fuzzy_orig_title'] = fuzzy_match_partial(train['orig_search_term'], train['orig_product_title'])
train['part_fuzzy_orig_descr'] = fuzzy_match_partial(train['orig_search_term'], train['orig_descr'])

## test
test['part_fuzzy_title'] = fuzzy_match_partial(test['search_term'], test['product_title'])
test['part_fuzzy_descr'] = fuzzy_match_partial(test['search_term'], test['descr'])

test['part_fuzzy_orig_title'] = fuzzy_match_partial(test['orig_search_term'], test['orig_product_title'])
test['part_fuzzy_orig_descr'] = fuzzy_match_partial(test['orig_search_term'], test['orig_descr'])

In [9]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

In [10]:
clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
clf.fit(train[use_cols])
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

In [13]:
gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

{'match_product_title': 244, 'match_descr': 179, 'len_search_term_words': 218, 'len_search_term_letters': 588, 'match_pos_title': 169, 'match_brand': 54, 'match_pos_descr': 175, 'prod_title_ind': 581, 'descr_ind': 480, 'words_std_title': 235, 'words_std_descr': 275, 'len_title': 377, 'len_descr': 386, 'last_word_title': 92, 'last_word_descr': 66, 'match_orig_title': 149, 'match_orig_descr': 88, 'fuzzy_title': 594, 'fuzzy_descr': 378, 'fuzzy_orig_title': 538, 'fuzzy_orig_descr': 371, 'part_fuzzy_title': 174, 'part_fuzzy_descr': 127, 'part_fuzzy_orig_title': 219, 'part_fuzzy_orig_descr': 76}


In [14]:
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.46886

In [18]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
pd.set_option('display.max_colwidth', -1)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff
id,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,95,60,60,57,57,100,100,100,100,1.143156
193001,183692,lichtenberg white no. 918 millenni ryan heather textur sheer curtain panel 40in. w xbi 95in. l,w g 918,3.00,w g 918,"LICHTENBERG White No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. W x 95 in. L","no. 918 millenni ryan heather textur semi-sh curtain casual solid add fresh finish touch decor setting. enhanc privaci allow light gentl filter through. clean, simpl one-pocket pole top design use standard decor curtain rod. mix match solid print look own.sheer panel, gentl filter lightno header pole top panelmachin washablewid array color choos from100% polyestercontain 1-curtain panel","No. 918 Millennial Ryan heathered texture semi-sheer curtain is a casual solid that adds freshness and a finishing touch to any decor setting. Enhances privacy while allowing light to gently filter through. Clean, simple one-pocket pole top design can be used with a standard or decorative curtain rod. Mix and match with other solids and prints for a look that is all your own.Sheer panel, gently filters lightNo header pole top panelMachine washableWide array of colors to choose from100% polyesterContains 1-curtain panel",0.666667,0.333333,...,56,83,60,83,60,100,100,100,100,1.084406
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden e

In [19]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,len_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff
id,,,,,,,,,,,,,,,,,,,,,
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"whether boil pasta, make batch chili cook grains, large, 10 quart stockpot right size. when stovetop, two short side handl help take less space cabinet. the hard-anod construct prefer even heat distribut throughout pan sides. the nonstick cook surfac mean need worri food stick scrub dirti pan - cleanup breeze. in fact, rachael stockpot dishwash safe ad convenience.hard-anod cookwar except durable, plus heat quick evenly, reduc ""hot spots"" burn foodsa clear coat exterior make cookwar dishwash safe conveniencelong-last nonstick interior let food slide eas make cleanup quick easyfun rubber handl provid comfort grip oven safe 350 degre fahrenheittemp glass lid watch food cook without remov lid keep heat moistur lock inlifetim limit warranti","Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience.Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foodsA clear coat exterior makes this cookware dishwasher safe for convenienceLong-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easyFun rubberized handles provide a comfortable grip and are oven safe to 350 degrees FahrenheitTempered glass lid to watch your foods cook without having to remove the lid to keep heat and moisture locked inLifetime limited warranty",1.000000,0.500000,...,113,100,78,100,78,100,100,100,100,-1.481658
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"enhanc home romano boxwood spiral topiari tree. this wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. you alway look forward rich green tone romano boxwood spiral topiari tree home. if look high qualiti artifici tree make bold statement home, it.light mobil designov 700 leavesindoor/outdooriron frameuv resist rubber leaves7 in. potenh room hous","Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",1.000000,1.000000,...,57,100,100,100,100,100,100,100,100,-1.452102
85851,125582,oakland live 26in. metal grape tabl plant stand,grape plant,1.00,grape plant,Oakland Living 26 in. Metal Grape Table Plant Stand,"the oakland live 26 in. metal grape tabl plant stand offer sturdi beauti place keep plant display. this stand made cast iron durabl antiqu bronz powder coat finish.mad cast ironantiqu bronz harden powder coat finish ad durabilityresist fading, chip crackingeasi follow assembl instruct hardwar includedstainless steel brass assembl hardwarefade, chip crack resistantsom assembl required1 year limit manufactur warrantyoth item avail collect","The O

In [17]:
train.to_csv('zip/train4.csv.gz', compression='gzip')
test.to_csv('zip/test4.csv.gz', compression='gzip')

In [34]:
train[train['orig_product_title'].str.find('DISCONTINUED') > -1][list(train.columns[:8]) + ['pred_diff']]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,pred_diff
id,,,,,,,,,
1038,100173,haier brewmast singl tap beer dispens black discontinu,beer keg dispens,3.00,beer keg dispenser,Haier BrewMaster Single Tap Beer Dispenser in Black-DISCONTINUED,"this singl tap beer dispens make conveni tast addit home bar, give great tast beer tap whenev want, comfort home. the unit counter height line kitchen counter featur studi worktop excel hold snack handl light job like slice limes. with abil dispens 1 mini barrel keg, one 1/4 barrel keg, one 1/2 keg, brewmast fit meet entertain person needs.singl tap brewmast dispens one mini barrel keg, one 1/4 barrel keg one 1/2 kegcomplet american sanki tap systemchrom beer tower & guardrail present polish lookheavi duti worktop & cabinetadjust thermostat keep beer optim drink temperaturefreestand applicationscast easi mobilitytap system clean kitconvert 6.4 cu. ft. beverag centerinclud 2 full-width wire shelvesco2 tank includ (co2 gas included)level legsclass all-black unit","This single tap beer dispenser makes a convenient and tasteful addition to your home bar, giving you the great taste of a beer on tap whenever you want, from the comfort of your own home. The unit is counter height to line up with kitchen counters and features a study worktop excellent for holding snacks or handling light jobs like slicing limes. With the ability to dispense 1 Mini Barrel Keg, one 1/4 Barrel Keg, or one 1/2 Keg, the BrewMaster is fit to meet all your entertaining and personal needs.Single Tap BrewMaster dispenses one mini barrel keg, one 1/4 barrel keg or one 1/2 KegComplete american sanky tap systemChrome beer tower & guardrail present a polished lookHeavy duty worktop & cabinetAdjustable Thermostat to keep beer at optimal drinking temperatureFreestanding ApplicationsCasters for easy mobilityTap system cleaning kitConverts to 6.4 cu. ft. Beverage CenterIncludes 2 Full-Width wire shelvesCO2 Tank Included (CO2 Gas not included)Leveling LegsClassic all-black unit",0.343998
9482,101614,nostalgia electr 6.0 cu. ft. kegor beer keg fridg dispens red discontinu,beer keg dispens,3.00,beer keg dispenser,Nostalgia Electrics 6.0 cu. ft. Kegorator Beer Keg Fridge Dispenser in Red-DISCONTINUED,"the nostalgia electr keg8000retror retro seri kegor beer keg fridg dispens beer easili quick even thirstiest crew. swivel caster make mobil enough roll around clubhouse, pool area anywher want get parti started. the includ 2.5 lb. co2 bottl good four 15 gal. keg fridg featur semi-gloss black tap tower spring loaded, top-mount tapper. a chrome guard rail drip tray finish look durabl red counter top. keep track amount co2 tank adjust pressur includ doubl meter regulator. this product ideal home brewer relax home.hold one full size 1/2 barrel keg, one 1/4 barrel poni size keg two 5 gal. d system kegskeg coupler fit d system valv fit non-standard keg sizesadjust thermostat interior temperatur rang unit 30-40 finclud empti 2.5-lb. co2 bottle, doubl meter regul drip trayspr load tap easi pull releas dispensingfour caster provid easi portability13 in. chrome beer tower guardrailcan convert regular refrigerator1-smal 1-larg wire shelveslock front caster","The Nostalgia Electrics KEG8000RETRORED Retro Series Kegorator Beer Keg Fridge dispenses beer easily and quickly to even the thirstiest crew. Swivel casters make it mobile enough to roll around the clubhouse, pool area or anywhere you want to get the party started. The included 2.5 lb. CO2 bottle is good for about four 15 gal. kegs and the fridge features a semi-gloss black tap tower with spring loaded, top-mounted tapper. A chrome guard rail and drip tray finishes the look of the durable red counter top. Keep track of the amount of CO2 in the tank or adjust pressure with the included double meter regulator. This product is ideal for home brewers or relaxing at home.Holds one full size 1/2 barrel keg, one 1/4 barrel pony size keg or two 5 

In [43]:
brand = pd.read_csv('zip/brand.csv.gz', index_col=['product_uid'], compression='gzip')
u_brands = brand['value'].unique()

@np.vectorize
def match_any_brand(search_term):
    if search_term in u_brands:
        return 1
    return 0

In [44]:
train['match_any_brand'] = match_any_brand(train['orig_search_term'])
train[train['match_any_brand'] == 1][list(train.columns[:8]) + ['brand', 'pred_diff']]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,brand,pred_diff
id,,,,,,,,,,
544,100089,melnor adjust rear trigger nozzl,melnor,3.00,melnor,Melnor Adjustable Rear Trigger Nozzle,this heavi duti melnor adjust rear trigger nozzl featur adjust tip spray mist jet tackl varieti water jobs. the nozzl trigger lock featur allow continu spraying. adjust tip featur mist jet spray varieti water jobs.assembl dimension: 1.38 in. w x 3.5 in. d x 5.38 in. hheavi duti zinc bodi construct metal nozzl durabilityinsul rubber handletrigg locksqueez handl easi control water flow,This heavy duty Melnor Adjustable Rear Trigger Nozzle features an Adjustable tip that sprays from mist to jet so you can tackle a variety of watering jobs. The nozzle's trigger lock feature allows for continuous spraying. Adjustable tip features a mist or jet spray for a variety of watering jobs.Assembled dimension: 1.38 in. W x 3.5 in. D x 5.38 in. HHeavy duty zinc body construction with metal nozzle for durabilityInsulated rubber handleTrigger lockSqueeze handle for easy control of water flow,melnor,0.188005
818,100141,closetmaid 36in. lamin 2 door rais panel storag cabinet white,closetmaid,2.67,closetmaid,ClosetMaid 36 in. Laminated 2-Door Raised Panel Storage Cabinet in White,"closetmaid 36 in. 2-door rais panel storag cabinet offer solut laundri room, garag area home need addit storage. secur 72 in. h x 36 in. w x 18.625 in. d cabinet wall enclos hardware. this provid extra safeti stability. the cabinet sculpt rais panel door brush nickel-pl handl add upscal look room.lamin wood materi durabl add contemporari feeldoor resist scratches, stains, impact, water uv rays2-fix shelv 2-adjust shelv versatil storag optionsinclud hardwar easy-to-follow assembl instructionseasi cleanup damp cloth","ClosetMaid's 36 in. 2-Door Raised Panel Storage Cabinet offers a solution for your laundry room, garage or other area of your home where you need additional storage. Secure this 72 in. H x 36 in. W x 18.625 in. D cabinet to the wall with the enclosed hardware. This will provide extra safety and stability. The cabinet's sculpted raised panel doors and brushed nickel-plated handles add an upscale look to any room.Laminate and wood material that is durable and adds a contemporary feelDoors that is resistant to scratches, stains, impact, water and UV rays2-fixed shelves and 2-adjustable shelves for versatile storage optionsIncluded hardware and easy-to-follow assembly instructionsEasy cleanup with just a damp cloth",closetmaid,0.076910
1076,100182,sheetrock firecod core 5 8in. xbi 4ft. xbi 12ft. gypsum board,sheetrock,3.00,sheetrock,Sheetrock Firecode Core 5/8 in. x 4 ft. x 12 ft. Gypsum Board,"sheetrock brand firecod core 5/8 in. x 4 ft. x 12 ft. gypsum board offer easi instal almost immedi decoration. panel easi score snap. their fire-resist gypsum core provid addit fire resist regular panels.intend fire-r applicationsth non-combust core encas 100% recycl face back papertap edg easili conceal jointsquick instal decorationresist crack warpingus new, repair remodel constructionnote: product may vari store.how much need? let calcul help:","Sheetrock Brand Firecode Core 5/8 in. x 4 ft. x 12 ft. Gypsum Board offers easy installation and almost immediate decoration. Panels are easy to score and snap. Their fire-resistant gypsum core provides additional fire resistance over regular panels.Intended for fire-rated applicationsThe non-combustible core is encased in 100% recycled face and back paperTapered edges for easily concealed jointsQuick installation and decorationResist cracking and warpingUse in new, or repair and remodel constructionNote: Product may vary by store.How much do you need? Let our calculator help:",sheetrock,0.164912
1206,100208,1 2in. brass csst xbi mipt termin flang,home flex,1.33,home-flex,1/2 in. Brass CSST x MIPT Termination Flange,"the home-flex termin flang provid smooth transit csst standard pipe. tool-less flare 

In [45]:
test['match_any_brand'] = match_any_brand(test['orig_search_term'])

In [46]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'match_any_brand']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

def fitter():
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

In [47]:
clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

In [48]:
gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

{'match_product_title': 271, 'match_descr': 200, 'len_search_term_words': 231, 'len_search_term_letters': 607, 'match_pos_title': 162, 'match_brand': 52, 'match_pos_descr': 163, 'prod_title_ind': 548, 'descr_ind': 432, 'words_std_title': 240, 'words_std_descr': 260, 'len_title': 383, 'len_descr': 345, 'last_word_title': 93, 'last_word_descr': 75, 'match_orig_title': 148, 'match_orig_descr': 114, 'fuzzy_title': 600, 'fuzzy_descr': 423, 'fuzzy_orig_title': 572, 'fuzzy_orig_descr': 333, 'part_fuzzy_title': 187, 'part_fuzzy_descr': 135, 'part_fuzzy_orig_title': 222, 'part_fuzzy_orig_descr': 78, 'match_any_brand': 3}


In [49]:
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.46930
Way off

In [51]:
@np.vectorize
def match_first_word(search_term, text):
    last_word = search_term.split()[0]
    if text.find(last_word) > -1:
        return 1
    else:
        return 0

train['search_term'] = train['search_term'].astype(str)
train['first_word_title'] = match_first_word(train['search_term'], train['product_title'])  
train['first_word_descr'] = match_first_word(train['search_term'], train['descr']) 

test['search_term'] = test['search_term'].astype(str)
test['first_word_title'] = match_first_word(test['search_term'], test['product_title'])  
test['first_word_descr'] = match_first_word(test['search_term'], test['descr']) 

tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

{'match_product_title': 259, 'match_descr': 190, 'len_search_term_words': 201, 'len_search_term_letters': 626, 'match_pos_title': 177, 'match_brand': 52, 'match_pos_descr': 184, 'prod_title_ind': 582, 'descr_ind': 501, 'words_std_title': 256, 'words_std_descr': 274, 'len_title': 397, 'len_descr': 350, 'last_word_title': 105, 'last_word_descr': 78, 'match_orig_title': 145, 'match_orig_descr': 106, 'fuzzy_title': 571, 'fuzzy_descr': 388, 'fuzzy_orig_title': 515, 'fuzzy_orig_descr': 360, 'part_fuzzy_title': 183, 'part_fuzzy_descr': 133, 'part_fuzzy_orig_title': 191, 'part_fuzzy_orig_descr': 93, 'first_word_title': 112, 'first_word_descr': 51}


In [52]:
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

In [66]:
@np.vectorize
def match_numbers(search_term, text):
    search_term_numbers = set(re.findall(r'\d+', search_term))
    len_ = len(search_term_numbers)
    if len_ == 0:
        return 0
    text_numbers = set(re.findall(r'\d+', text))
    match = len([1 for i in search_term_numbers if i in text_numbers])
    return match / len_
    

In [67]:
train['match_numbers_title'] = match_numbers(train['search_term'], train['product_title'])  
train['match_numbers_descr'] = match_numbers(train['search_term'], train['descr']) 

test['match_numbers_title'] = match_numbers(test['search_term'], test['product_title'])  
test['match_numbers_descr'] = match_numbers(test['search_term'], test['descr']) 

In [68]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'match_numbers_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

{'match_product_title': 257, 'match_descr': 197, 'len_search_term_words': 216, 'len_search_term_letters': 635, 'match_pos_title': 167, 'match_brand': 57, 'match_pos_descr': 152, 'prod_title_ind': 579, 'descr_ind': 459, 'words_std_title': 247, 'words_std_descr': 273, 'len_title': 395, 'len_descr': 360, 'last_word_title': 108, 'last_word_descr': 83, 'match_orig_title': 94, 'match_orig_descr': 99, 'fuzzy_title': 584, 'fuzzy_descr': 432, 'fuzzy_orig_title': 537, 'fuzzy_orig_descr': 386, 'part_fuzzy_title': 187, 'part_fuzzy_descr': 134, 'part_fuzzy_orig_title': 214, 'part_fuzzy_orig_descr': 78, 'first_word_title': 88, 'first_word_descr': 58, 'match_numbers_title': 163, 'match_numbers_descr': 21}


In [69]:
y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

0.46879

In [71]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 244, 'match_descr': 169, 'len_search_term_words': 243, 'len_search_term_letters': 635, 'match_pos_title': 174, 'match_brand': 59, 'match_pos_descr': 187, 'prod_title_ind': 614, 'descr_ind': 477, 'words_std_title': 235, 'words_std_descr': 263, 'len_title': 403, 'len_descr': 362, 'last_word_title': 88, 'last_word_descr': 82, 'match_orig_title': 92, 'match_orig_descr': 115, 'fuzzy_title': 554, 'fuzzy_descr': 394, 'fuzzy_orig_title': 522, 'fuzzy_orig_descr': 385, 'part_fuzzy_title': 184, 'part_fuzzy_descr': 126, 'part_fuzzy_orig_title': 194, 'part_fuzzy_orig_descr': 87, 'first_word_title': 107, 'first_word_descr': 69, 'match_numbers_title': 133}


0.46750

In [70]:
@np.vectorize
def has_number(search_term):
    search_term_numbers = set(re.findall(r'\d+', search_term))
    if len(search_term_numbers) == 0:
        return 0
    return 1
    

In [73]:
train['has_number_search_term'] = has_number(train['search_term'])
train['has_number_title'] = has_number(train['product_title'])  
train['has_number_descr'] = has_number(train['descr']) 

test['has_number_search_term'] = has_number(test['search_term'])
test['has_number_title'] = has_number(test['product_title'])  
test['has_number_descr'] = has_number(test['descr']) 

In [74]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'has_number_search_term', 'has_number_title', 'has_number_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 252, 'match_descr': 172, 'len_search_term_words': 181, 'len_search_term_letters': 687, 'match_pos_title': 169, 'match_brand': 52, 'match_pos_descr': 159, 'prod_title_ind': 568, 'descr_ind': 466, 'words_std_title': 242, 'words_std_descr': 253, 'len_title': 403, 'len_descr': 351, 'last_word_title': 84, 'last_word_descr': 78, 'match_orig_title': 82, 'match_orig_descr': 119, 'fuzzy_title': 596, 'fuzzy_descr': 385, 'fuzzy_orig_title': 541, 'fuzzy_orig_descr': 341, 'part_fuzzy_title': 181, 'part_fuzzy_descr': 133, 'part_fuzzy_orig_title': 237, 'part_fuzzy_orig_descr': 90, 'first_word_title': 95, 'first_word_descr': 46, 'match_numbers_title': 158, 'has_number_search_term': 119, 'has_number_title': 45, 'has_number_descr': 15}


0.46775

In [75]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 'has_number_search_term']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 260, 'match_descr': 181, 'len_search_term_words': 206, 'len_search_term_letters': 663, 'match_pos_title': 167, 'match_brand': 61, 'match_pos_descr': 155, 'prod_title_ind': 516, 'descr_ind': 441, 'words_std_title': 272, 'words_std_descr': 284, 'len_title': 384, 'len_descr': 363, 'last_word_title': 90, 'last_word_descr': 65, 'match_orig_title': 83, 'match_orig_descr': 108, 'fuzzy_title': 565, 'fuzzy_descr': 437, 'fuzzy_orig_title': 539, 'fuzzy_orig_descr': 373, 'part_fuzzy_title': 157, 'part_fuzzy_descr': 123, 'part_fuzzy_orig_title': 219, 'part_fuzzy_orig_descr': 79, 'first_word_title': 85, 'first_word_descr': 57, 'match_numbers_title': 167, 'has_number_search_term': 127}


0.46805

backuping files

In [76]:
train.columns

Index(['product_uid', 'product_title', 'search_term', 'relevance',
       'orig_search_term', 'orig_product_title', 'descr', 'orig_descr',
       'match_product_title', 'match_descr', 'len_search_term_words',
       'len_search_term_letters', 'complete_match_title',
       'complete_match_descr', 'brand', 'match_brand', 'words_std_title',
       'words_std_descr', 'last_word_title', 'last_word_descr',
       'prod_title_ind', 'descr_ind', 'match_pos_title', 'match_pos_descr',
       'match_orig_title', 'match_orig_descr', 'len_title', 'len_descr',
       'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
       'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title',
       'part_fuzzy_orig_descr', 'pred_diff', 'first_word_title',
       'first_word_descr', 'match_numbers_title', 'match_numbers_descr',
       'has_number_search_term', 'has_number_title', 'has_number_descr'],
      dtype='object')

In [77]:
train = train.drop(columns=['has_number_search_term', 'has_number_title', 'has_number_descr'])
test = test.drop(columns=['has_number_search_term', 'has_number_title', 'has_number_descr'])
train.to_csv('zip/train4.csv.gz', compression='gzip')
test.to_csv('zip/test4.csv.gz', compression='gzip')

In [78]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
pd.set_option('display.max_colwidth', -1)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,57,100,100,100,100,1.122377,0,0,0,0
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510replaces: dantona 3aa, empir cpb-400, general electr bt-17, panason p-p341pa, recoton t121, uniden ba-3003.6-volt / 1200mahnickel-met hydride1 year warranti","Power your Panasonic, American Telecom, Tele-Phone, Key Phone, Audiovox, Uniden, Universal, and Zeta cordless phones with Lenmar's Nickel-Metal Hydride cordless phone replacement battery. This 3.6-Volt/1200mAh battery will provide reliable, long-lasting power. Compatible with Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, and Zeta 510.Fits: Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, Zeta 510Replaces: Dantona 3AA, Empire CPB-400, General Electric BT-17, Panasonic P-P341PA, Recoton T121, Uniden BA-3003.6-volt / 1200mAhNickel-metal hydride1 year warranty",0.666667,0.333333,...,74,100,100,100,100,1.114812,0,0,0,0
193001,183692,lichtenberg white no. 918 millenni ryan heather textur sheer curtain panel 40in. w xbi 95in. l,w g 918,3.00,w g 918,"LICHTENBERG White No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. 

In [79]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,pred_diff,first_word_title,first_word_descr,match_numbers_title,match_numbers_descr
id,,,,,,,,,,,,,,,,,,,,,
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"enhanc home romano boxwood spiral topiari tree. this wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. you alway look forward rich green tone romano boxwood spiral topiari tree home. if look high qualiti artifici tree make bold statement home, it.light mobil designov 700 leavesindoor/outdooriron frameuv resist rubber leaves7 in. potenh room hous","Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",1.000000,1.000000,...,100,100,100,100,100,-1.465766,1,1,0,0
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"whether boil pasta, make batch chili cook grains, large, 10 quart stockpot right size. when stovetop, two short side handl help take less space cabinet. the hard-anod construct prefer even heat distribut throughout pan sides. the nonstick cook surfac mean need worri food stick scrub dirti pan - cleanup breeze. in fact, rachael stockpot dishwash safe ad convenience.hard-anod cookwar except durable, plus heat quick evenly, reduc ""hot spots"" burn foodsa clear coat exterior make cookwar dishwash safe conveniencelong-last nonstick interior let food slide eas make cleanup quick easyfun rubber handl provid comfort grip oven safe 350 degre fahrenheittemp glass lid watch food cook without remov lid keep heat moistur lock inlifetim limit warranti","Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience.Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foodsA clear coat exterior makes this cookware dishwasher safe for convenienceLong-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easyFun rubberized handles provide a comfortable grip and are oven safe to 350 degrees FahrenheitTempered glass lid to watch your foods cook without having to remove the lid to keep heat and moisture locked inLifetime limited warranty",1.000000,0.500000,...,78,100,100,100,100,-1.412411,1,1,0,0
156918,160006,werner 14ft. fiberglass round rung straight ladder 375lb. load capac type iaa duti rate,14ft. ladder,1.00,14 ft ladder,Werner 14 ft. Fiberglass Round Rung Straight Ladder with 375 lb. Load Capacity Type IAA Duty Rating,"the 7114-1 one-sect round rung 14 ft. straight ladder made non-conduct fiberglass rails. easi transport maneuver, ladder bear duti rate 375 lbs. make durabl long lasting. the slip-resistant, traction-tr rung round, durabl rail shield shoe bracket help protect rail damage. the plate rung assembl rivet four point side rails.alflo twist-proof perfor

In [80]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 
             'complete_match_title', 'complete_match_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 225, 'match_descr': 176, 'len_search_term_words': 180, 'len_search_term_letters': 651, 'match_pos_title': 157, 'match_brand': 52, 'match_pos_descr': 175, 'prod_title_ind': 536, 'descr_ind': 525, 'words_std_title': 221, 'words_std_descr': 267, 'len_title': 374, 'len_descr': 411, 'last_word_title': 102, 'last_word_descr': 80, 'match_orig_title': 94, 'match_orig_descr': 109, 'fuzzy_title': 625, 'fuzzy_descr': 363, 'fuzzy_orig_title': 568, 'fuzzy_orig_descr': 368, 'part_fuzzy_title': 171, 'part_fuzzy_descr': 136, 'part_fuzzy_orig_title': 196, 'part_fuzzy_orig_descr': 95, 'first_word_title': 100, 'first_word_descr': 69, 'match_numbers_title': 134, 'complete_match_title': 38, 'complete_match_descr': 12}


0.46800

In [81]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title', 
             'complete_match_title']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 251, 'match_descr': 180, 'len_search_term_words': 236, 'len_search_term_letters': 631, 'match_pos_title': 152, 'match_brand': 66, 'match_pos_descr': 171, 'prod_title_ind': 550, 'descr_ind': 441, 'words_std_title': 236, 'words_std_descr': 261, 'len_title': 397, 'len_descr': 345, 'last_word_title': 87, 'last_word_descr': 65, 'match_orig_title': 91, 'match_orig_descr': 112, 'fuzzy_title': 559, 'fuzzy_descr': 387, 'fuzzy_orig_title': 560, 'fuzzy_orig_descr': 343, 'part_fuzzy_title': 160, 'part_fuzzy_descr': 136, 'part_fuzzy_orig_title': 224, 'part_fuzzy_orig_descr': 81, 'first_word_title': 85, 'first_word_descr': 60, 'match_numbers_title': 157, 'complete_match_title': 39}


0.46797

In [82]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

y_pred = gbm.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm5.csv', header=True, index=False)

{'match_product_title': 225, 'match_descr': 164, 'len_search_term_words': 245, 'len_search_term_letters': 608, 'match_pos_title': 160, 'match_brand': 56, 'match_pos_descr': 137, 'prod_title_ind': 586, 'descr_ind': 461, 'words_std_title': 248, 'words_std_descr': 257, 'len_title': 399, 'len_descr': 340, 'last_word_title': 106, 'last_word_descr': 80, 'match_orig_title': 89, 'match_orig_descr': 124, 'fuzzy_title': 577, 'fuzzy_descr': 383, 'fuzzy_orig_title': 523, 'fuzzy_orig_descr': 323, 'part_fuzzy_title': 201, 'part_fuzzy_descr': 133, 'part_fuzzy_orig_title': 201, 'part_fuzzy_orig_descr': 78, 'first_word_title': 104, 'first_word_descr': 75, 'match_numbers_title': 139}


0.46747